In [1]:
from sys import path
path.insert(0,'../../python')
from google_api import *
from map_utils import *
import folium
from folium.plugins import MarkerCluster,HeatMap, Fullscreen
from folium import plugins
from async_request import make_request
import json

In [2]:
begin = "ROBERT BOSCH MEXICO Santa Fe CDMX"
end = "Orizaba 32 CDMX"

# geocode_result = gmaps.geocode(begin)
# bosch_location = get_geocode_locations(geocode_result)[0]
bosch_location = [19.3675809, -99.26231600000001]

# geocode_result = gmaps.geocode(end)
# orizaba_location = get_geocode_locations(geocode_result)[0]
orizaba_location = [19.4225152, -99.1606875]

### Test

In [3]:
from datetime import datetime, timedelta
time = datetime.now()
time = datetime(2020,2,18, 19,20,0)

directions_result = gmaps.directions(begin,
                                     end,
                                     departure_time= time,
                                     mode="transit",
                                     alternatives=True,
                                     )

In [4]:
start_time = datetime(2020,2,24, 5,0,0)
end_time = datetime(2020,2,24, 22,0,0)
current_time = start_time
monday_times = []
while current_time <= end_time:
    monday_times.append(current_time)
    current_time += timedelta(minutes = 15)
    
start_time = datetime(2020,2,26, 5,0,0)
end_time = datetime(2020,2,26, 22,0,0)    
current_time = start_time
wednesday_times = []
while current_time <= end_time:
    wednesday_times.append(current_time)
    current_time += timedelta(minutes = 15)
    
start_time = datetime(2020,2,28, 5,0,0)
end_time = datetime(2020,2,28, 22,0,0)    
current_time = start_time
friday_times = []
while current_time <= end_time:
    friday_times.append(current_time)
    current_time += timedelta(minutes = 15)
    
monday_data_way1 = [[begin,end, time] for time in monday_times]
monday_data_way2 = [[end,begin, time] for time in monday_times]
wednesday_data_way1 = [[begin,end, time] for time in wednesday_times]
wednesday_data_way2 = [[end,begin, time] for time in wednesday_times]
friday_data_way1 = [[begin,end, time] for time in friday_times]
friday_data_way2 = [[end,begin, time] for time in friday_times]

In [5]:
def fetch_transit(session, data, i):
    start, end, time = data
    try:
        response = gmaps.directions(start, 
                                 end, 
                                 mode="transit", 
                                 alternatives=False, 
                                 departure_time=time)
        return response
    except Exception as e:
        print('Error trying to request the directions', e)
        return []

```python
monday_routes_way1 = make_request(fetch_traffic, monday_data_way1)
monday_routes_way2 = make_request(fetch_traffic, monday_data_way2)
wednesday_routes_way1 = make_request(fetch_traffic, wednesday_data_way1)
wednesday_routes_way2 = make_request(fetch_traffic, wednesday_data_way2)
friday_routes_way1 = make_request(fetch_traffic, friday_data_way1)
friday_routes_way2 = make_request(fetch_traffic, friday_data_way2)

transit_routes = {
    'monday_routes_way1':monday_routes_way1,
    'monday_routes_way2':monday_routes_way2,
    'wednesday_routes_way1':wednesday_routes_way1,
    'wednesday_routes_way2':wednesday_routes_way2,
    'friday_routes_way1':friday_routes_way1,
    'friday_routes_way2':friday_routes_way2
}


with open('../../data/Orizaba 32/transit_routes.json', 'w') as f:
    json.dump(transit_routes, f)
```

In [6]:
with open('../../data/Orizaba 32/transit_routes.json', 'r') as f:
        transit_routes = json.load(f)
        
monday_routes_way1 = transit_routes['monday_routes_way1']
monday_routes_way2 = transit_routes['monday_routes_way2']
wednesday_routes_way1 = transit_routes['wednesday_routes_way1']
wednesday_routes_way2 = transit_routes['wednesday_routes_way2']
friday_routes_way1 = transit_routes['friday_routes_way1']
friday_routes_way2 = transit_routes['friday_routes_way2']

In [7]:
def hours_minutes(td):
    return td.seconds//3600, (td.seconds//60)%60

In [8]:

def get_routes_info(routes):
    durations = []
    arrivals = []
    departures = []
    for route in routes:
        legs = route[0]['legs'][0]
        departure = datetime.fromtimestamp(legs['departure_time']['value'])
        arrival = datetime.fromtimestamp(legs['arrival_time']['value'])
        td = arrival - departure
        duration = ':'.join(str(td).split(':')[:2])
        arrival = ':'.join(str(arrival).split(' ')[1].split(':')[:2])
        departure = ':'.join(str(departure).split(' ')[1].split(':')[:2])
        durations.append(duration)
        arrivals.append(arrival)
        departures.append(departure)
    return departures, durations,arrivals

In [18]:
import pandas as pd

In [21]:
departures, durations, arrivals = get_routes_info(monday_routes_way1)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_mon1.csv')

In [22]:
departures, durations, arrivals = get_routes_info(monday_routes_way1)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_mon1.csv')

In [23]:
departures, durations, arrivals = get_routes_info(monday_routes_way2)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_mon2.csv')

In [24]:
departures, durations, arrivals = get_routes_info(wednesday_routes_way1)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_wed1.csv')

In [25]:
departures, durations, arrivals = get_routes_info(wednesday_routes_way2)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_wed2.csv')

In [26]:
departures, durations, arrivals = get_routes_info(friday_routes_way1)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_fri1.csv')

In [27]:
departures, durations, arrivals = get_routes_info(friday_routes_way2)

df = pd.DataFrame({
    'departures': departures,
    'durations':durations,
    'arrivals':arrivals
             })

df.to_csv('../../data/Orizaba 32/transit_fri2.csv')

In [29]:
df

,departures,durations,arrivals
0,05:26,1:00,06:27
1,05:26,1:00,06:27
2,05:30,1:00,06:30
3,05:45,1:00,06:45
4,06:00,1:00,07:00
...,...,...,...
64,21:00,1:05,22:05
65,21:15,1:05,22:20
66,21:30,1:05,22:35
67,21:45,1:05,22:50


In [28]:
route = directions_result[3]
def print_transit_route(route):
    print(f"Price: {route['fare']['text']}")
    print(f"Departure time: {route['legs'][0]['departure_time']['text']}")
    print(f"Arrival time: {route['legs'][0]['arrival_time']['text']}")
    print(f"Duration: {route['legs'][0]['duration']['text']}")
    print(f"Distance: {route['legs'][0]['distance']['text']}")

In [20]:
directions_result[0]['legs'][0].keys()

dict_keys(['arrival_time', 'departure_time', 'distance', 'duration', 'end_address', 'end_location', 'start_address', 'start_location', 'steps', 'traffic_speed_entry', 'via_waypoint'])

In [22]:
directions_result[0]['legs'][0].keys()

dict_keys(['arrival_time', 'departure_time', 'distance', 'duration', 'end_address', 'end_location', 'start_address', 'start_location', 'steps', 'traffic_speed_entry', 'via_waypoint'])

In [38]:
len(directions_result[0]['legs'][0]['steps'])

3

In [46]:
steps[0].keys()

dict_keys(['distance', 'duration', 'end_location', 'html_instructions', 'polyline', 'start_location', 'steps', 'travel_mode'])

In [47]:
steps[0]['travel_mode']

'WALKING'

In [42]:
steps[1].keys()

dict_keys(['distance', 'duration', 'end_location', 'html_instructions', 'polyline', 'start_location', 'transit_details', 'travel_mode'])

In [43]:
steps[1]['travel_mode']

'TRANSIT'

In [44]:
steps[1]['transit_details']

{'arrival_stop': {'location': {'lat': 19.4245476, 'lng': -99.15869289999999},
  'name': 'Avenida Chapultepec - Mérida'},
 'arrival_time': {'text': '12:16pm',
  'time_zone': 'America/Mexico_City',
  'value': 1582049809},
 'departure_stop': {'location': {'lat': 19.3724878, 'lng': -99.2593315},
  'name': 'Avenida Vasco de Quiroga - Roberto Medellin'},
 'departure_time': {'text': '11:37am',
  'time_zone': 'America/Mexico_City',
  'value': 1582047470},
 'headsign': 'Cierre de circuito Santa Fe - Metro Balderas',
 'headway': 300,
 'line': {'agencies': [{'name': 'Red de Transporte de Pasajeros',
    'url': 'https://www.rtp.cdmx.gob.mx/'}],
  'color': '#91f244',
  'name': 'ECOBÚS Santa Fe - Metro Balderas',
  'short_name': '34-A',
  'text_color': '#000000',
  'vehicle': {'icon': '//maps.gstatic.com/mapfiles/transit/iw2/6/bus2.png',
   'name': 'Bus',
   'type': 'BUS'}},
 'num_stops': 26}

In [34]:
steps[2].keys()

dict_keys(['distance', 'duration', 'end_location', 'html_instructions', 'polyline', 'start_location', 'steps', 'travel_mode'])

In [63]:
tiles_url = 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png'
attrb ='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>'
m = folium.Map(tiles=None)

folium.TileLayer(tiles_url,attr=attrb,name = 'Orizaba 32 - Car', show = True).add_to(m)


text = '<p><b>Bosch Santa Fe</b></p>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)
folium.Marker(bosch_location, icon = folium.Icon(color='red'), popup=popup).add_to(m)

text = '<p><b>Orizaba 32</b></p>'
text = folium.Html(text, script=True)
popup = folium.Popup(text, max_width=2650)
folium.Marker(orizaba_location, icon = folium.Icon(color='blue'), popup=popup).add_to(m)

for i, route in enumerate(directions_result):
    lyr = folium.FeatureGroup(name = f'Ruta {i + 1}',show=True if i == 0 else False)
    steps = route['legs'][0]['steps']
    for step in steps:
        color = 'gray' if step['travel_mode'] == 'WALKING' else 'green'
        folium.PolyLine(decode_polyline(step['polyline']['points']), color = color, weight = 6, popup=None).add_to(lyr)
        #folium.PolyLine(decode_polyline(traffic_polyline), color = 'red', weight = 6, popup=None).add_to(traffic_lyr)


    lyr.add_to(m)
    
    
Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m.fit_bounds(m.get_bounds())
m
#m.save('../maps/Bosch Anstatt.html')

In [46]:
legs['steps'][1]

{'distance': {'text': '12.9 km', 'value': 12941},
 'duration': {'text': '39 mins', 'value': 2339},
 'end_location': {'lat': 19.4245476, 'lng': -99.15869289999999},
 'html_instructions': 'Bus towards Cierre de circuito Santa Fe - Metro Balderas',
 'polyline': {'points': 'aufuBxqy|QILSQg@g@MM_@a@UQKKe@e@g@g@_@]m@k@a@]QQ[[CCWYCEIKSYKOUa@_AsBwA_DWi@ISQk@Og@Oc@O_@ISUe@Ue@iB}DMWSe@CIGMCIEKEQEOAEUcAIg@G]ESMk@Oo@Me@CMUy@Ok@I[W_AEOESU_AKk@EWCSLGHKHKDMDM@OAOAOAEAEAECCAECCCECCGEGEGEICIAIAI?G?E@E@E@E@E@CBEBC@EFEDEDCFCFAB?BAFAFIRGHKJ]NaAXODiAXODSDWF[FUDSBS@W?Q?OCMA]GSIKGIGICIGECi@g@IESUU]W_@KQGGw@sASWGE[OcBuCm@oAIOISCIa@_ASe@IUIWEOOe@Ma@O_@Yk@]e@OOUUWQs@_@UMgCqAMIw@e@WUAAIIIKQYQ_@GEECa@{@GMSa@a@o@e@k@QQ]a@W_@MSO[ACWi@y@mBOYIQMQWYQQOIMKQI_@Qu@Wg@MuFqA]GMCMAM?OAs@AgA@_@?S?KAGAGAKEMEQIGCIGOMQOKIQQUYMSGMKUKYCQCMC[?A?EAG?U@g@@c@B]B]?U?Q?WA[Ea@EQG[Ma@Ma@Oc@GWUs@_@mAM]s@_Cg@cBOe@EMEOESCOAUAQAW?U@YFwA@c@?WAMCa@CSCOQq@I]Qs@Ka@Kg@QiAE_@EY[oCAGY}BKw@?ACUAO?QAK?O?O@S?QNmBDo@D_A@u@?k@Ac@Ai@Cg@Cg@Eg@C]Gi@CSIm@CI

In [56]:
for route in wednesday_routes_way2:
    legs = route[0]['legs'][0]
    for step in legs['steps']:
        if 'transit_details' in step:
            transit_details = step['transit_details']
            line = transit_details['line']
            print(transit_details['departure_stop']['name'],' -.-', transit_details['arrival_stop']['name'])

Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Napoles  -.- Antonio Dovali Jaime - Avenida Santa Fe
Avenida Chapultepec - Nap

In [41]:
34-A ECOBÚS Santa Fe - Metro Balderas

5